#Setting up project

In [ ]:
from xgboost import XGBClassifier
from sklearn import svm
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn import metrics

In [ ]:
# load csv data file
from google.colab import files
uploaded = files.upload()

Saving CS3244_parsed_data full.csv to CS3244_parsed_data full.csv


In [ ]:
# Read and prepare data
import io
df = pd.read_csv(io.BytesIO(uploaded['CS3244_parsed_data full.csv']))

Initial testings

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features_all = ['avg', 'vol', 'avg7', 'sd7', 'range7', 'vol7',  'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
labels = ['dirNext1', 'dirNext7','dirNext30']
count_1 = 0
count_0 = 0
count_2 = 0

for index, row in df.iterrows():
  if (isinstance(row['date'], str)):
    dt = datetime.strptime(row['date'], '%d/%m/%Y')
    curr = []

    date = int(dt.strftime('%Y-%m-%d').translate({ord('-'): None}))
    curr.append(date)

    for feature in features_all:
      curr.append(row[feature])

    if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
      x_test.append(curr)
      y_test_7.append(row['dirNext7'])
      y_test_30.append(row['dirNext30'])
    else: # train data
      x_train.append(curr)
      y_train_7.append(row['dirNext7'])
      y_test_30.append(row['dirNext30'])

    if (row['dirNext7'] == 1): 
        count_1 += 1
    elif (row['dirNext7'] == 0):
        count_0 += 1
    elif (row['dirNext7'] == -1):
        count_2 += 1

print(count_1)
print(count_0)
print(count_2)



99639
72852
69584


In [ ]:
bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
print(y_pred_7)

[-1  1 -1 ...  1  1 -1]


In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test_7, y_pred_7))
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))

Accuracy: 0.423216885007278
[[ 2873  2485 11064]
 [ 1595  3970 10265]
 [ 2243  4048 16417]]
              precision    recall  f1-score   support

          -1       0.43      0.17      0.25     16422
           0       0.38      0.25      0.30     15830
           1       0.43      0.72      0.54     22708

    accuracy                           0.42     54960
   macro avg       0.41      0.38      0.36     54960
weighted avg       0.42      0.42      0.39     54960



#Feature reduction using df7

found best set of features : ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']

## removed sd7

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'range7', 'vol7',  'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = ['avg7', 'sd7', 'range7', 'vol7',  'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


## removed vol7

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'range7', 'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['range7', 'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.42194323144104806
[[ 2911  2367 11144]
 [ 1563  3887 10380]
 [ 2300  4016 16392]]
              precision    recall  f1-score   support

          -1       0.43      0.18      0.25     16422
           0       0.38      0.25      0.30     15830
           1       0.43      0.72      0.54     22708

    accuracy                           0.42     54960
   macro avg       0.41      0.38      0.36     54960
weighted avg       0.42      0.42      0.38     54960

testing features: 
['avg7', 'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.4229803493449782
[[ 5562  4850 22432]
 [ 3074  7622 20964]
 [ 4264  7842 33310]]
              precision    recall  f1-score   support

          -1       0.43      0.17      0.24     32844
           0       0.38      0.24      0.29     31660
           1     

## removed range7, accuracy 0.4229803493449782

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['dir7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.4170487627365357
[[ 2924  2537 10961]
 [ 1779  3841 10210]
 [ 2497  4055 16156]]
              precision    recall  f1-score   support

          -1       0.41      0.18      0.25     16422
           0       0.37      0.24      0.29     15830
           1       0.43      0.71      0.54     22708

    accuracy                           0.42     54960
   macro avg       0.40      0.38      0.36     54960
weighted avg       0.41      0.42      0.38     54960

testing features: 
['avg7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.42337154294032026
[[ 6065  4915 21864]
 [ 3640  7594 20426]
 [ 4733  7805 32878]]
              precision    recall  f1-score   support

          -1       0.42      0.18      0.26     32844
           0       0.37      0.24      0.29     31660
           1       0.44      0.72  

## removed dir 7, accuracy 0.42337154294032026

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.4217066957787482
[[ 2706  2314 11402]
 [ 1584  3623 10623]
 [ 2169  3691 16848]]
              precision    recall  f1-score   support

          -1       0.42      0.16      0.24     16422
           0       0.38      0.23      0.28     15830
           1       0.43      0.74      0.55     22708

    accuracy                           0.42     54960
   macro avg       0.41      0.38      0.36     54960
weighted avg       0.41      0.42      0.38     54960

testing features: 
['avg7', 'avg30', 'sd30', 'range30', 'vol30', 'dir30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.4144650655021834
[[ 3894  4758 24192]
 [ 2416  7542 21702]
 [ 3647  7647 34122]]
              precision    recall  f1-score   support

          -1       0.39      0.12      0.18     32844
           0       0.38      0.24      0.29     31660
           1       0.43      0.75      0.54     45416

## removed dir30, accuracy 0.422325327510917

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'tend30', 'RSI', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.4210698689956332
[[ 2811  2401 11210]
 [ 1712  3836 10282]
 [ 2310  3903 16495]]
              precision    recall  f1-score   support

          -1       0.41      0.17      0.24     16422
           0       0.38      0.24      0.30     15830
           1       0.43      0.73      0.54     22708

    accuracy                           0.42     54960
   macro avg       0.41      0.38      0.36     54960
weighted avg       0.41      0.42      0.38     54960

testing features: 
['avg7', 'avg30', 'sd30', 'range30', 'vol30', 'tend30', 'RSI', 'stosc', 'MACD']
Accuracy: 0.4182496360989811
[[ 4288  4464 24092]
 [ 2516  7441 21703]
 [ 3790  7381 34245]]
              precision    recall  f1-score   support

          -1       0.40      0.13      0.20     32844
           0       0.39      0.24      0.29     31660
           1       0.43      0.75      0.55     45416

    accuracy    

## removed RSI, accuracy 0.423598981077147

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'tend30', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'tend30', 'stosc', 'MACD']
Accuracy: 0.4262008733624454
[[ 2830  2266 11326]
 [ 1629  3609 10592]
 [ 2285  3438 16985]]
              precision    recall  f1-score   support

          -1       0.42      0.17      0.24     16422
           0       0.39      0.23      0.29     15830
           1       0.44      0.75      0.55     22708

    accuracy                           0.43     54960
   macro avg       0.41      0.38      0.36     54960
weighted avg       0.42      0.43      0.38     54960

testing features: 
['avg7', 'avg30', 'sd30', 'range30', 'vol30', 'tend30', 'stosc', 'MACD']
Accuracy: 0.42279839883551673
[[ 4150  4633 24061]
 [ 2507  7666 21487]
 [ 3553  7205 34658]]
              precision    recall  f1-score   support

          -1       0.41      0.13      0.19     32844
           0       0.39      0.24      0.30     31660
           1       0.43      0.76      0.55     45416

    accuracy                 

## removed trend30, accuracy 0.42796059471823666

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['avg7', 'tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']
Accuracy: 0.4314410480349345
[[ 2998  2193 11231]
 [ 1729  3548 10553]
 [ 2376  3166 17166]]
              precision    recall  f1-score   support

          -1       0.42      0.18      0.25     16422
           0       0.40      0.22      0.29     15830
           1       0.44      0.76      0.56     22708

    accuracy                           0.43     54960
   macro avg       0.42      0.39      0.37     54960
weighted avg       0.42      0.43      0.39     54960

testing features: 
['avg7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']
Accuracy: 0.42648289665211064
[[ 4499  3876 24469]
 [ 2488  6373 22799]
 [ 3566  5843 36007]]
              precision    recall  f1-score   support

          -1       0.43      0.14      0.21     32844
           0       0.40      0.20      0.27     31660
           1       0.43      0.79      0.56     45416

    accuracy                           0.43    10

## removed avg7, accuracy 0.4314410480349345

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']


KeyboardInterrupt: ignored

## removed sd30, accuracy 0.42970038816108685

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['tend7', 'avg30', 'range30', 'vol30', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['avg30', 'range30', 'vol30', 'stosc', 'MACD']
Accuracy: 0.4181222707423581
[[ 1695  2026 12701]
 [ 1083  3310 11437]
 [ 1609  3124 17975]]
              precision    recall  f1-score   support

          -1       0.39      0.10      0.16     16422
           0       0.39      0.21      0.27     15830
           1       0.43      0.79      0.55     22708

    accuracy                           0.42     54960
   macro avg       0.40      0.37      0.33     54960
weighted avg       0.40      0.42      0.36     54960

testing features: 
['tend7', 'range30', 'vol30', 'stosc', 'MACD']
Accuracy: 0.4265465793304221
[[ 4326  4246 24272]
 [ 2428  6736 22496]
 [ 3282  6310 35824]]
              precision    recall  f1-score   support

          -1       0.43      0.13      0.20     32844
           0       0.39      0.21      0.28     31660
           1       0.43      0.79      0.56     45416

    accuracy                           0.43    109920
   macro avg       0.42      

## removed vol30, accuracy 0.4267148835516739

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['tend7', 'avg30', 'range30', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['avg30', 'range30', 'stosc', 'MACD']
Accuracy: 0.4193231441048035
[[ 1821  2217 12384]
 [ 1199  3708 10923]
 [ 1740  3451 17517]]
              precision    recall  f1-score   support

          -1       0.38      0.11      0.17     16422
           0       0.40      0.23      0.29     15830
           1       0.43      0.77      0.55     22708

    accuracy                           0.42     54960
   macro avg       0.40      0.37      0.34     54960
weighted avg       0.41      0.42      0.36     54960

testing features: 
['tend7', 'range30', 'stosc', 'MACD']
Accuracy: 0.4287754730713246
[[ 4488  4028 24328]
 [ 2584  6652 22424]
 [ 3532  5893 35991]]
              precision    recall  f1-score   support

          -1       0.42      0.14      0.21     32844
           0       0.40      0.21      0.28     31660
           1       0.43      0.79      0.56     45416

    accuracy                           0.43    109920
   macro avg       0.42      0.38      0.35    

## removed avg30, accuracy 0.4287754730713246
## removed range30, accuracy 0.42457241630276565

In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['tend7', 'range30', 'stosc', 'MACD']
count = len(features)
max = 0
max_features = []

# for y in range(2):
for x in range(len(features)): 
  features_curr = features[:x] + features[x+1:]
  print("testing features: ")
  print(features_curr)

  for index, row in df.iterrows():
    if (isinstance(row['date'], str)):
      dt = datetime.strptime(row['date'], '%d/%m/%Y')
      curr = []

      for feature in features_curr:
        curr.append(row[feature])

      if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
        x_test.append(curr)
        # print(curr)
        y_test_7.append(row['dirNext7'])
      else: # train data
        x_train.append(curr)
        y_train_7.append(row['dirNext7'])

  bst = XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
  bst.fit(np.array(x_train), np.array(y_train_7))
  y_pred_7 = bst.predict(x_test)
  accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
  print("Accuracy:", accuracy)
  print(confusion_matrix(y_test_7,y_pred_7))
  print(classification_report(y_test_7,y_pred_7))

  if (accuracy > max): 
    max = accuracy
    max_features = features_curr

print("------------------ end -----------")
print("max accuracy:")
print(max)
print(max_features)
max = 0
features = max_features
print("------------------ end -----------")


testing features: 
['range30', 'stosc', 'MACD']
Accuracy: 0.4211972343522562
[[ 1292  2178 12952]
 [  711  3491 11628]
 [ 1085  3257 18366]]
              precision    recall  f1-score   support

          -1       0.42      0.08      0.13     16422
           0       0.39      0.22      0.28     15830
           1       0.43      0.81      0.56     22708

    accuracy                           0.42     54960
   macro avg       0.41      0.37      0.32     54960
weighted avg       0.41      0.42      0.35     54960

testing features: 
['tend7', 'stosc', 'MACD']
Accuracy: 0.42457241630276565
[[ 4096  4023 24725]
 [ 2171  5861 23628]
 [ 2762  5942 36712]]
              precision    recall  f1-score   support

          -1       0.45      0.12      0.20     32844
           0       0.37      0.19      0.25     31660
           1       0.43      0.81      0.56     45416

    accuracy                           0.42    109920
   macro avg       0.42      0.37      0.34    109920
weighted avg

Conclusion: 
Set of features yielding best accuracy is ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']

# Testing different models


1.   n_estimators
2.   max_depth
3.   learning_rate





In [ ]:
x_train = []
y_train_7 = []
x_test = []
y_test_7 = []
features = ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']

for index, row in df.iterrows():
  if (isinstance(row['date'], str)):
    dt = datetime.strptime(row['date'], '%d/%m/%Y')
    curr = []

    for feature in features:
      curr.append(row[feature])

    if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
      x_test.append(curr)
      y_test_7.append(row['dirNext7'])
    else: # train data
      x_train.append(curr)
      y_train_7.append(row['dirNext7'])

testing combinations of n_estimators and max_depth

In [ ]:
  max_x = -1
  max_y = -1
  max_accuracy = 0
  
  for x in range(1, 10):
    for y in range(1, 10): 
      print("testing", 10*x, " ", 10*y)
      bst = XGBClassifier(n_estimators=10*x, max_depth=10*y, learning_rate=1, objective='binary:logistic', tree_method='hist')
      bst.fit(np.array(x_train), np.array(y_train_7))
      y_pred_7 = bst.predict(x_test)
      accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
      print("Accuracy:", accuracy)
      print(confusion_matrix(y_test_7,y_pred_7))
      print(classification_report(y_test_7,y_pred_7))

      if (accuracy > max_accuracy):
        max_x = 10*x
        max_y = 10*y
        max_accuracy = accuracy

testing 10   10
Accuracy: 0.3941957787481805
[[ 4337  3474  8611]
 [ 3465  4582  7783]
 [ 4729  5233 12746]]
              precision    recall  f1-score   support

          -1       0.35      0.26      0.30     16422
           0       0.34      0.29      0.31     15830
           1       0.44      0.56      0.49     22708

    accuracy                           0.39     54960
   macro avg       0.38      0.37      0.37     54960
weighted avg       0.38      0.39      0.38     54960

testing 10   20
Accuracy: 0.37565502183406113
[[ 4902  4105  7415]
 [ 4188  4877  6765]
 [ 5734  6107 10867]]
              precision    recall  f1-score   support

          -1       0.33      0.30      0.31     16422
           0       0.32      0.31      0.32     15830
           1       0.43      0.48      0.46     22708

    accuracy                           0.38     54960
   macro avg       0.36      0.36      0.36     54960
weighted avg       0.37      0.38      0.37     54960

testing 10   30
Acc

In [ ]:
print (max_x)
print (max_y)
print (max_accuracy)

10
10
0.3941957787481805


In [ ]:
  max_x = -1
  max_y = -1
  max_accuracy = 0
  
  for x in range(10):
    for y in range(10): 
      print("testing", x+2, " ", y+2)
      bst = XGBClassifier(n_estimators=x+2, max_depth=y+2, learning_rate=1, objective='binary:logistic', tree_method='hist')
      bst.fit(np.array(x_train), np.array(y_train_7))
      y_pred_7 = bst.predict(x_test)
      accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
      print("Accuracy:", accuracy)
      print(confusion_matrix(y_test_7,y_pred_7))
      print(classification_report(y_test_7,y_pred_7))

      if (accuracy > max_accuracy):
        max_x = x+2
        max_y = y+2
        max_accuracy = accuracy

testing 2   2
Accuracy: 0.41584788937409023
[[  319   113 15990]
 [  204   319 15307]
 [  226   265 22217]]
              precision    recall  f1-score   support

          -1       0.43      0.02      0.04     16422
           0       0.46      0.02      0.04     15830
           1       0.42      0.98      0.58     22708

    accuracy                           0.42     54960
   macro avg       0.43      0.34      0.22     54960
weighted avg       0.43      0.42      0.26     54960

testing 2   3
Accuracy: 0.42467248908296945
[[ 1423  1553 13446]
 [  730  2347 12753]
 [  950  2188 19570]]
              precision    recall  f1-score   support

          -1       0.46      0.09      0.15     16422
           0       0.39      0.15      0.21     15830
           1       0.43      0.86      0.57     22708

    accuracy                           0.42     54960
   macro avg       0.42      0.37      0.31     54960
weighted avg       0.42      0.42      0.34     54960

testing 2   4
Accuracy

In [ ]:
print (max_x)
print (max_y)
print (max_accuracy)

3
5
0.4320778748180495


best combination when n_estimators=3, max_depth=5

-------------------------

testing combinations of learning_rates

In [ ]:
bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))

bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=0.9, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))


bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1.1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))


bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1.2, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))

bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=0.8, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))


Accuracy: 0.4320778748180495
[[ 2588  2025 11809]
 [ 1396  3248 11186]
 [ 1867  2930 17911]]
              precision    recall  f1-score   support

          -1       0.44      0.16      0.23     16422
           0       0.40      0.21      0.27     15830
           1       0.44      0.79      0.56     22708

    accuracy                           0.43     54960
   macro avg       0.43      0.38      0.36     54960
weighted avg       0.43      0.43      0.38     54960

Accuracy: 0.4308951965065502
[[ 2309  1860 12253]
 [ 1159  3009 11662]
 [ 1569  2775 18364]]
              precision    recall  f1-score   support

          -1       0.46      0.14      0.22     16422
           0       0.39      0.19      0.26     15830
           1       0.43      0.81      0.57     22708

    accuracy                           0.43     54960
   macro avg       0.43      0.38      0.35     54960
weighted avg       0.43      0.43      0.37     54960

Accuracy: 0.42918486171761283
[[ 2707  1959 11756]
 

Conclusion: model works well with n_estimators=3, max_depth=5, learning_rate=1 setting

# Testing with different training sets

1.   changing 2008 to test data
2.   removing 2008 from test and train data

Conclusion: does not seem to improve accuracy result

In [ ]:
x_train = []
y_train_7 = []
x_test = []
y_test_7 = []
features = ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']

for index, row in df.iterrows():
  if (isinstance(row['date'], str)):
    dt = datetime.strptime(row['date'], '%d/%m/%Y')
    curr = []

    for feature in features:
      curr.append(row[feature])

    if (dt.year == 2009 or dt.year == 2012 or dt.year == 2017): # test data
      x_test.append(curr)
      y_test_7.append(row['dirNext7'])
    else: # train data
      x_train.append(curr)
      y_train_7.append(row['dirNext7'])

bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))

Accuracy: 0.42787268346587926
[[ 2502  2575  8796]
 [ 1892  3908 11350]
 [ 2934  4035 17209]]
              precision    recall  f1-score   support

          -1       0.34      0.18      0.24     13873
           0       0.37      0.23      0.28     17150
           1       0.46      0.71      0.56     24178

    accuracy                           0.43     55201
   macro avg       0.39      0.37      0.36     55201
weighted avg       0.40      0.43      0.39     55201



In [ ]:
x_train = []
y_train_7 = []
x_test = []
y_test_7 = []
features = ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']

for index, row in df.iterrows():
  if (isinstance(row['date'], str)):
    dt = datetime.strptime(row['date'], '%d/%m/%Y')
    curr = []

    for feature in features:
      curr.append(row[feature])

    if (dt.year == 2017): # test data
      x_test.append(curr)
      y_test_7.append(row['dirNext7'])
    elif (dt.year == 2013 or dt.year == 2014 or dt.year == 2015 or dt.year == 2016): # train data
      x_train.append(curr)
      y_train_7.append(row['dirNext7'])

bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))

Accuracy: 0.4222941720629047
[[ 724  699 2297]
 [ 896 1219 3534]
 [1182 1384 5361]]
              precision    recall  f1-score   support

          -1       0.26      0.19      0.22      3720
           0       0.37      0.22      0.27      5649
           1       0.48      0.68      0.56      7927

    accuracy                           0.42     17296
   macro avg       0.37      0.36      0.35     17296
weighted avg       0.40      0.42      0.39     17296



# Testing with df30 using different set of features

In [ ]:
x_train = []
y_train_30 = []
y_train_7 = []
x_test = []
y_test_30 = []
y_test_7 = []
features = ["open", "close", "high", "low", "avg", "avg7", "range7", "avg30",  "range30"] 
for index, row in df.iterrows():
  if (isinstance(row['date'], str)):
    dt = datetime.strptime(row['date'], '%d/%m/%Y')
    curr = []

    for feature in features:
      curr.append(row[feature])

    date = int(dt.strftime('%Y-%m-%d').translate({ord('-'): None}))
    curr.append(date)

    if (dt.year == 2008 or dt.year == 2012 or dt.year == 2017): # test data
      x_test.append(curr)
      y_test_30.append(row['dirNext30'])
      y_test_7.append(row['dirNext7'])
    else: # train data
      x_train.append(curr)
      y_train_30.append(row['dirNext30'])
      y_train_7.append(row['dirNext7'])

In [ ]:
bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_30))
y_pred_30 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_30, y_pred_30)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_30,y_pred_30))
print(classification_report(y_test_30,y_pred_30))

Accuracy: 0.5426491994177584
[[ 8092   237  5749]
 [ 3471   843 11042]
 [ 4204   433 20889]]
              precision    recall  f1-score   support

          -1       0.51      0.57      0.54     14078
           0       0.56      0.05      0.10     15356
           1       0.55      0.82      0.66     25526

    accuracy                           0.54     54960
   macro avg       0.54      0.48      0.43     54960
weighted avg       0.54      0.54      0.47     54960



In [ ]:
bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_7))
y_pred_7 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_7, y_pred_7)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_7,y_pred_7))
print(classification_report(y_test_7,y_pred_7))

Accuracy: 0.442085152838428
[[ 7506   308  8608]
 [ 3320   712 11798]
 [ 6062   567 16079]]
              precision    recall  f1-score   support

          -1       0.44      0.46      0.45     16422
           0       0.45      0.04      0.08     15830
           1       0.44      0.71      0.54     22708

    accuracy                           0.44     54960
   macro avg       0.44      0.40      0.36     54960
weighted avg       0.44      0.44      0.38     54960



# more testings after consulting TA
*  Testing higher n_estimators, max_depth, and smaller learning_rate
*   List item



In [ ]:
x_train = []
y_train_7 = []
y_train_30 = []
x_test = []
y_test_7 = []
y_test_30 = []
features = ['tend7', 'avg30', 'sd30', 'range30', 'vol30', 'stosc', 'MACD']

for index, row in df.iterrows():
  if (isinstance(row['date'], str)):
    dt = datetime.strptime(row['date'], '%d/%m/%Y')
    curr = []

    for feature in features:
      curr.append(row[feature])

    if (dt.year == 2009 or dt.year == 2012 or dt.year == 2017): # test data
      x_test.append(curr)
      y_test_7.append(row['dirNext7'])
      y_test_30.append(row['dirNext30'])
    else: # train data
      x_train.append(curr)
      y_train_7.append(row['dirNext7'])
      y_train_30.append(row['dirNext30'])

In [ ]:
bst = XGBClassifier(n_estimators=100, max_depth=7, learning_rate=0.1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_30))
y_pred_30 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_30, y_pred_30)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_30,y_pred_30))
print(classification_report(y_test_30,y_pred_30))

Accuracy: 0.4777449683882538
[[  903  1238  6943]
 [ 1163  3139 11715]
 [ 2966  4804 22330]]
              precision    recall  f1-score   support

          -1       0.18      0.10      0.13      9084
           0       0.34      0.20      0.25     16017
           1       0.54      0.74      0.63     30100

    accuracy                           0.48     55201
   macro avg       0.36      0.35      0.34     55201
weighted avg       0.43      0.48      0.44     55201



In [ ]:
df = df.rename(columns = {'Unnamed: 0' :'index'})
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
train_year = [2005, 2006, 2007, 2008, 2010, 2011, 2013, 2014, 2015, 2016]
test_year = [2009, 2012, 2017]

df_train = df[df['year'].isin(train_year)]
df_test = df[df['year'].isin(test_year)]

from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# based on corr matrix, choose those 4 features with lowest corr
df30_Xtrain = df_train[["avg30", "sd30", "range30", "vol30", "dir30", "tend30", "RSI", "stosc", "MACD"]].copy() 
df30_ytrain = df_train['dirNext30'].copy()
df30_Xtest = df_test[["avg30", "sd30", "range30", "vol30", "dir30", "tend30", "RSI", "stosc", "MACD"]].copy()
df30_ytest = df_test['dirNext30'].copy()  

In [ ]:
bst = XGBClassifier(n_estimators=3, max_depth=5, learning_rate=0.1, objective='binary:logistic', tree_method='hist')
bst.fit(np.array(x_train), np.array(y_train_30))
y_pred_30 = bst.predict(x_test)
accuracy = metrics.accuracy_score(y_test_30, y_pred_30)
print("Accuracy:", accuracy)
print(confusion_matrix(y_test_30,y_pred_30))
print(classification_report(y_test_30,y_pred_30))

Accuracy: 0.5415481603594138
[[  122   108  8854]
 [  133   341 15543]
 [  374   295 29431]]
              precision    recall  f1-score   support

          -1       0.19      0.01      0.03      9084
           0       0.46      0.02      0.04     16017
           1       0.55      0.98      0.70     30100

    accuracy                           0.54     55201
   macro avg       0.40      0.34      0.26     55201
weighted avg       0.46      0.54      0.40     55201



# conclusion

Due to time constraint, we did not explore more possibilities of using boosting trees. Our final performance of boosting trees was similar to random forest.

References: 

XGBoost documentation: https://xgboost.readthedocs.io/en/stable/tutorials/multioutput.html

Introduction to boosted trees: https://towardsdatascience.com/introduction-to-boosted-trees-2692b6653b53#:~:text=Boosting%20transforms%20weak%20decision%20trees,known%20as%20ensemble%20meta%2Dalgorithms.

